<a href="https://colab.research.google.com/github/dho619/TCC_Pos_IA/blob/main/TratamentoDosDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
df = pd.read_csv('./Walmart.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


In [31]:
#transformando o campo de date que estava como object, para datetime
df.Date=pd.to_datetime(df.Date)

#separando a data em 3 colunas (dia, mês e ano)
df['Day'] = df.Date.dt.day
df['Month'] = df.Date.dt.month
df['Year'] = df.Date.dt.year

#removendo a coluna de data original
df.drop(['Date'], axis=1, inplace=True)

df.head

<bound method NDFrame.head of       Store  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price         CPI  \
0         1    1643690.90             0        42.31       2.572  211.096358   
1         1    1641957.44             1        38.51       2.548  211.242170   
2         1    1611968.17             0        39.93       2.514  211.289143   
3         1    1409727.59             0        46.63       2.561  211.319643   
4         1    1554806.68             0        46.50       2.625  211.350143   
...     ...           ...           ...          ...         ...         ...   
6430     45     713173.95             0        64.88       3.997  192.013558   
6431     45     733455.07             0        64.89       3.985  192.170412   
6432     45     734464.36             0        54.47       4.000  192.327265   
6433     45     718125.53             0        56.47       3.969  192.330854   
6434     45     760281.43             0        58.85       3.882  192.308899   

      Une

In [32]:
#Fazendo uma cópia para manter o Data Frame original
original_df = df.copy(deep=True);

In [33]:
#pegando o shape
rows,cols = original_df.shape;

#removendo linhas duplicadas
df.drop_duplicates(inplace=True);

if df.shape==(rows,cols):
    print("Não existe linhas duplicadas");
else:
    print("Foram removidas {} linhas duplicadas!".format(rows-df.shape[0]));

Não existe linhas duplicadas


In [34]:
#Verificando se existe valores null, nesse caso não tem
df.isnull().sum().sort_values()

Store           0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
Day             0
Month           0
Year            0
dtype: int64

In [35]:
#Ignorar as colunas
ignoreFeatures = ['Weekly_Sales', 'day'];
#variável para definir até quantos valores unicos uma coluna pode ter e ainda considerada categorical
maximumToBeCategorical = 45; #45 por ser o total de lojas

#pegar as colunas, menos as ignoradas
features = [i for i in df.columns if i not in ignoreFeatures]

#pega a quantidade de valores unicos para cara coluna
uniqueValuesForFeature = df[features].nunique().sort_values()

#Adiciona nas lista se for uma coluna de categorias ou de valores
categoricalFeature = [];
valuesFeature = [];
for i in range(df[features].shape[1]):
    if uniqueValuesForFeature.values[i]<=maximumToBeCategorical:
        categoricalFeature.append(uniqueValuesForFeature.index[i])
    else:
        valuesFeature.append(uniqueValuesForFeature.index[i])
print(categoricalFeature)
print(valuesFeature)

['Holiday_Flag', 'Year', 'Month', 'Day', 'Store']
['Unemployment', 'Fuel_Price', 'CPI', 'Temperature']


In [36]:
df2 = df.copy()

#Transforma cada valor unico das colunas selecionadas, em novas colunas
#e seta como true ou false, de acordo com o valor original
for i in categoricalFeature:
    if df2[i].nunique()==2:
        df2[i]=pd.get_dummies(df2[i], drop_first=True, prefix=str(i))
    if (df2[i].nunique()>2):
        df2 = pd.concat([df2.drop([i], axis=1), pd.DataFrame(pd.get_dummies(df2[i], prefix=str(i)))],axis=1)
        
df2.shape

(6435, 97)

In [37]:
df3 = df2.copy()

#Apenas analisando os possíveis outlers e se necessário deletar
for i in valuesFeature:
    Q1 = df3[i].quantile(0.25)
    Q3 = df3[i].quantile(0.75)
    IQR = Q3 - Q1
    #Calculos comumente aceitos para encontrar outliers
    print("Coluna {} tem {} outliers (para cima)".format(i, rows - df3[df3[i] <= (Q3+(1.5*IQR))].shape[0]))
    print("Coluna {} tem {} outliers (para baixo)".format(i, rows - df3[df3[i] >= (Q1-(1.5*IQR))].shape[0]))
    #remover os outliers (nesse caso, foi opitado por não fazer)
    #df3 = df3[df3[i] <= (Q3+(1.5*IQR))]
    #df3 = df3[df3[i] >= (Q1-(1.5*IQR))]
    #df3 = df3.reset_index(drop=True) #resetar os index
'''
 Como a maioria dos outliers estão relacionadas a taxa de desemprego e os demais na temperatura,
 decidi por manter os registros para evitar tirar o enfluenciamento dessas variáveis nas vendas
'''

Coluna Unemployment tem 378 outliers (para cima)
Coluna Unemployment tem 103 outliers (para baixo)
Coluna Fuel_Price tem 0 outliers (para cima)
Coluna Fuel_Price tem 0 outliers (para baixo)
Coluna CPI tem 0 outliers (para cima)
Coluna CPI tem 0 outliers (para baixo)
Coluna Temperature tem 0 outliers (para cima)
Coluna Temperature tem 3 outliers (para baixo)


'\n Como a maioria dos outliers estão relacionadas a taxa de desemprego e os demais na temperatura,\n decidi por manter os registros para evitar tirar o enfluenciamento dessas variáveis nas vendas\n'

In [38]:
#Apenas voltando a variavel para df
df = df3.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6435 entries, 0 to 6434
Data columns (total 97 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Weekly_Sales  6435 non-null   float64
 1   Holiday_Flag  6435 non-null   uint8  
 2   Temperature   6435 non-null   float64
 3   Fuel_Price    6435 non-null   float64
 4   CPI           6435 non-null   float64
 5   Unemployment  6435 non-null   float64
 6   Year_2010     6435 non-null   uint8  
 7   Year_2011     6435 non-null   uint8  
 8   Year_2012     6435 non-null   uint8  
 9   Month_1       6435 non-null   uint8  
 10  Month_2       6435 non-null   uint8  
 11  Month_3       6435 non-null   uint8  
 12  Month_4       6435 non-null   uint8  
 13  Month_5       6435 non-null   uint8  
 14  Month_6       6435 non-null   uint8  
 15  Month_7       6435 non-null   uint8  
 16  Month_8       6435 non-null   uint8  
 17  Month_9       6435 non-null   uint8  
 18  Month_10      6435 non-null 

In [39]:
df.to_csv("Walmart_tratado.csv")